In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 7.63 ms (started: 2021-08-13 02:21:41 +08:00)


In [3]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 #1 Aug 13, 2021'

git push origin master

origin	git@github.com:ustchope/kerastuner.git (fetch)
origin	git@github.com:ustchope/kerastuner.git (push)
[master 5d10e01] 更新 #1 Aug 13, 2021
 1 file changed, 25 insertions(+), 25 deletions(-)


To github.com:ustchope/kerastuner.git
   2610353..5d10e01  master -> master


time: 3.82 s (started: 2021-08-13 02:22:20 +08:00)


In [2]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 1.43 s (started: 2021-08-13 02:22:09 +08:00)


# 介绍

keras_tuner.engine.tuner.Tuner 中的 Tuner 类可以被子类化以支持高级用途，例如：
* 自定义训练循环（GAN、强化学习等）
* 在模型构建功能之外添加超参数（预处理、数据增强、测试时间增强等）

本教程不会介绍支持非 Keras 模型的子类化。 为此，您可以将 `keras_tuner.engine.base_tuner.BaseTuner` 类子类化（参见 `keras_tuner.tuners.sklearn.Sklearn` 示例）。

# 了解搜索过程

`Tuner.search` 可以传递任何参数。 这些参数将直接传递给 `Tuner.run_trial`，以及一个 `Trial` 对象，该对象包含有关当前试验的信息，包括超参数和试验状态。 通常，`Tuner.run_trial` 是用户在子类化 Tuner 时需要覆盖的唯一方法。

# 重载 run_trial

run_trial 有两种写法。 一种是利用 Tuner 的内置回调钩子，将目标值发送到 Oracle 并保存模型的最新状态。 这些钩子是：
* self.on_epoch_end：必须被调用。 将结果报告给 Oracle 并保存模型。 传递给此方法的日志字典必须包含目标名称。
* self.on_epoch_begin、self.on_batch_begin、self.on_batch_end：可选。 这些方法在 Tuner 中什么都不做，但是如果您希望子类的用户创建他们自己的子类来覆盖训练过程的这些部分，那么这些方法作为钩子提供是很有用的。

In [ ]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, ...):
        model = self.hypermodel.build(trial.hyperparameters)
        for epoch in range(10):
              epoch_loss = ...
              self.on_epoch_end(trial, model, epoch, logs={'loss': epoch_loss})

或者，您可以改为直接调用用于向 Oracle 报告结果并保存模型的方法。 对于没有自然时期概念或您不想在每个时期后向 Oracle 报告结果的用例，这可以提供更大的灵活性。 这些方法是：
* self.oracle.update_trial：向 Oracle 报告当前结果。 传递给此方法的指标字典必须包含目标名称。
* self.save_model：保存训练好的模型。

In [ ]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, ...):
        model = self.hypermodel.build(trial.hyperparameters)
        score = ...
        self.oracle.update_trial(trial.trial_id, {'score': score})
        self.save_model(trial.trial_id, model)

## 在预处理、评估等过程中添加超参数。

新的 HyperParameters 可以在 run_trial 的任何地方定义，就像在 HyperModel 中定义 HyperParameters 一样。 这些超参数在第一次遇到时采用默认值，然后由 Oracle 调整。

In [ ]:
class MyTuner(kt.Tuner):

    def run_trial(self, trial, ...):
        hp = trial.hyperparameters
        model = self.hypermodel.build(hp)

        batch_size = hp.Int('batch_size', 32, 128, step=32)
        random_flip = hp.Boolean('random_flip')

## 端到端示例：

In [4]:
import keras_tuner as kt
import tensorflow as tf
import numpy as np


def build_model(hp):
    """Builds a convolutional model."""
    inputs = tf.keras.Input(shape=(28, 28, 1))
    x = inputs
    for i in range(hp.Int("conv_layers", 1, 3, default=3)):
        x = tf.keras.layers.Conv2D(
            filters=hp.Int("filters_" + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
            activation="relu",
            padding="same",
        )(x)

        if hp.Choice("pooling" + str(i), ["max", "avg"]) == "max":
            x = tf.keras.layers.MaxPooling2D()(x)
        else:
            x = tf.keras.layers.AveragePooling2D()(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

    if hp.Choice("global_pooling", ["max", "avg"]) == "max":
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    optimizer = hp.Choice("optimizer", ["adam", "sgd"])
    model.compile(
        optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

time: 446 ms (started: 2021-08-13 02:32:13 +08:00)


In [ ]:
class MyTuner(kt.Tuner):
    def run_trial(self, trial, train_ds):
        hp = trial.hyperparameters

        # Hyperparameters can be added anywhere inside `run_trial`.
        # When the first trial is run, they will take on their default values.
        # Afterwards, they will be tuned by the `Oracle`.
        train_ds = train_ds.batch(hp.Int("batch_size", 32, 128, step=32, default=64))

        model = self.hypermodel.build(trial.hyperparameters)
        lr = hp.Float("learning_rate", 1e-4, 1e-2, sampling="log", default=1e-3)
        optimizer = tf.keras.optimizers.Adam(lr)
        epoch_loss_metric = tf.keras.metrics.Mean()
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        # @tf.function
        def run_train_step(data):
            images = tf.dtypes.cast(data[0], "float32") / 255.0
            labels = data[1]
            with tf.GradientTape() as tape:
                logits = model(images)
                loss = loss_fn(labels, logits)
                # Add any regularization losses.
                if model.losses:
                    loss += tf.math.add_n(model.losses)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            epoch_loss_metric.update_state(loss)
            return loss

        # `self.on_epoch_end` reports results to the `Oracle` and saves the
        # current state of the Model. The other hooks called here only log values
        # for display but can also be overridden. For use cases where there is no
        # natural concept of epoch, you do not have to call any of these hooks. In
        # this case you should instead call `self.oracle.update_trial` and
        # `self.oracle.save_model` manually.
        for epoch in range(2):
            print("Epoch: {}".format(epoch))

            self.on_epoch_begin(trial, model, epoch, logs={})
            for batch, data in enumerate(train_ds):
                self.on_batch_begin(trial, model, batch, logs={})
                batch_loss = float(run_train_step(data))
                self.on_batch_end(trial, model, batch, logs={"loss": batch_loss})

                if batch % 100 == 0:
                    loss = epoch_loss_metric.result().numpy()
                    print("Batch: {}, Average Loss: {}".format(batch, loss))

            epoch_loss = epoch_loss_metric.result().numpy()
            self.on_epoch_end(trial, model, epoch, logs={"loss": epoch_loss})
            epoch_loss_metric.reset_states()

In [ ]:
tuner = MyTuner(
    oracle=kt.oracles.BayesianOptimization(
        objective=kt.Objective("loss", "min"), max_trials=2
    ),
    hypermodel=build_model,
    directory="results",
    project_name="mnist_custom_training",
)

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))[:1000]
y_train = y_train.astype(np.int64)[:1000]

mnist_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))

tuner.search(train_ds=mnist_train)

best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

best_model = tuner.get_best_models()[0]